In [1]:
import pandas as pd
from utils import config
from utils.bert.data import *

from sklearn.model_selection import train_test_split

config.xls_path = '../Train-Data/Mix6_mainCat_20/pro_review.xlsx'
# config.xls_path = '/home/eagleuser/Users/leyan/Train-Data/Mix6_mainCat_new/pro_review.xlsx'

total_df = pd.read_excel(config.xls_path)
# total_df = total_df[total_df['review_len']>=50]
# total_df = total_df[total_df['summary_len']>=5]
total_df = total_df.sort_values(by=['review_len','overlap'], ascending = False)
train_df, val_df = train_test_split(total_df, test_size=0.1, 
                                    random_state=0, shuffle=True)
vocab = Vocab()
tokenizer = vocab.tokenizer

print(len(total_df))
print(len(train_df))
print(len(val_df))

vocab_file ../temp/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
{'BOS': 1, 'EOS': 2, 'PAD': 0, 'EOQ': 3, 'SEP': 102, 'CLS': 101, 'UNK': 100}
512937
461643
51294


In [2]:
def longest_common_subsequence(main_string, comparing_string):

#     main_string = main_string.split(" ")
#     comparing_string = comparing_string.split(" ")
    columns_length = len(main_string)  # Get the length of the first word or base word
    rows_length = len(comparing_string)  # Get the length of the second word or comparing word

    # MAKE A 2D LIST (MATRIX)
    dynamic_table = [[0] * (columns_length + 1) for i in range(rows_length + 1)]

    # rows_length = NUMBER OF ROWS
    # columns_length = NUMBER OF COLUMNS
    
    # FILL THE MATRIX FOLLOWING LCS ALGORITHM.
    for i in range(1, rows_length + 1):
        for j in range(1, columns_length + 1):
            if main_string[j - 1] == comparing_string[i - 1]:
                dynamic_table[i][j] = 1 + dynamic_table[i - 1][j - 1]

            else:
                dynamic_table[i][j] = max(dynamic_table[i - 1][j], dynamic_table[i][j - 1])

#     print("MATRIX ACCORDING TO LONGEST COMMON SUBSEQUENCE ALGORITHM: \n ")

    # for i in range(rows_length + 1):
    #     print(dynamic_table[i])

    #print("LENGTH OF LONGEST COMMON SUBSEQUENCE = ", dynamic_table[rows_length][columns_length])

    len_lcs = dynamic_table[rows_length][columns_length]

    i = len(comparing_string)
    j = len(main_string)

    lcs_string = str()

    # BACKTRACKING TO FIND THE LONGEST COMMON SUBSEQUENCE

    temp = True

    while temp is True:
        if dynamic_table[i][j] == 0:
            temp = False
        elif dynamic_table[i][j] == dynamic_table[i][j - 1]:
            j = j - 1

        else:
            lcs_string = main_string[j-1] + " " + lcs_string
            i = i - 1
            j = j - 1

    return lcs_string, len_lcs

In [3]:
# df.loc[i,'token_lcs'] = token_lcs
#             df.loc[i,'len_lcs'] = len_lcs
#             df.loc[i,'percent_lcs'] = percent_lcs
#             df.loc[i,'overlap_pos'] = overlap_pos
            
#             df.loc[i,'reveiw_len'] = len(rev_tokens)
#             df.loc[i,'summary_len'] = len(summary_tokens)

In [4]:
#%%
from tqdm import tqdm
from copy import deepcopy
def analay(df, mode):
    
    df['token_lcs'] = 0
    df['len_lcs'] = 0
    df['percent_lcs'] = 0
    df['overlap_pos'] = 0

    df['reveiw__len'] = 0
    df['summary_len'] = 0
#     df = df[:100]
    with tqdm(total=len(df)) as pbar:
        for i ,row in df.iterrows():       
            overlap_pos = None

            rev, summary = row['review'], row['summary']
            rev_tokens, summary_tokens = tokenizer.tokenize(rev), tokenizer.tokenize(summary)            
            token_lcs, len_lcs = longest_common_subsequence(rev_tokens, summary_tokens)
            percent_lcs = len_lcs / len(summary_tokens) * 100 # percent

            rev_sents = rev.split(" . ")
            overlap_sents = {idx: longest_common_subsequence(tokenizer.tokenize(sent), summary_tokens) for idx, sent in enumerate(rev_sents)}
            sort_overlap_sent = {k:v for k,v in sorted(overlap_sents.items(), key= lambda item: item[1], reverse=True)}
            overlap_sent_id = list(sort_overlap_sent.items())[0][0]

            if (overlap_sent_id == 0) and (percent_lcs >= 20): overlap_pos = 0 # overlap_Top 
            elif (overlap_sent_id == len(rev_tokens)-1) and (percent_lcs >= 20): overlap_pos = 1 # overlap_Final 
            elif (percent_lcs >= 20): overlap_pos = 2 # overlap_other 
            else: overlap_pos = -1 # no overlap 

            df.loc[i,'token_lcs'] = token_lcs
            df.loc[i,'len_lcs'] = len_lcs
            df.loc[i,'percent_lcs'] = percent_lcs
            df.loc[i,'overlap_pos'] = overlap_pos
            
            df.loc[i,'reveiw_len'] = len(rev_tokens)
            df.loc[i,'summary_len'] = len(summary_tokens)
            pbar.update(1)
            pbar.set_description("%s row: %s | overlap_pos_bert: %s (%2.2f)percent_bert| len_lcs_bert: %s" % (mode, i, overlap_pos, percent_lcs, len_lcs))
    return df



In [ ]:
train_df.head()

,review_ID,rating,vote,binaryrating,review,summary,cheat,cheat_num,overlap,review_len,summary_len,POS_keys,DEP_keys,Noun_adj_keys,TextRank_keys
240956,1422144000,2,3,negative,not buy these glove they are good . after one ...,<s> tear at the seam and hurt my knuckle </s>,"{'hurt', 'knuckle', 'seam', 'tear'}",4,6,73,10,"['buy', 'glove', '', 'crap', 'wear', '', 'read...","['buy', 'glove', 'crap', 'wear', 'hurt', 'knuc...","['glove', 'good', 'punching', 'class', 'hardco...","['like', 'hardcore', 'read']"
509083,1514851200,2,2,negative,buy this few month ago hold food . the plastic...,<s> lid does not click into place </s>,"{'click', 'place', 'lid'}",3,5,86,8,"['food', 'hold', 'size', 'correct', 'rating', ...","['ago', 'hold', 'month', 'food', 'few', 'size'...","['month', 'few', 'size', 'plastic', 'correct',...","['month', 'ago', 'hold', 'food', 'shut']"
274813,1405728000,4,2,positive,the bag just picture . well construct and has ...,<s> the bag is just as picture it is well </s>,"{'bag', 'well', 'picture'}",3,5,40,11,"['', 'adequate', 'storage', 'insulate', 'compa...","['adequate', 'has', 'storage', 'construct', 'b...","['storage', 'construct', 'adequate', 'compartm...","['adequate', 'storage', 'water']"
348041,1491955200,4,4,positive,purchase pair for year old son . little bigger...,<s> the cup holder is the best part besides th...,"{'great', 'padding', 'holder', 'best', 'cup'}",5,8,46,13,"['purchase', 'old', 'pair', 'son', '', '', '',...","['old', 'son', 'keep', 'cup', 'padding', 'grea...","['year', 'pair', 'son', 'old', 'padding', 'hol...","['purchase', 'pair']"
189227,1438128000,5,2,positive,wear orthotic take out the arch support that t...,<s> my foot do not slide around but are perfec...,"{'foot', 'fit', 'slide', 'hurt', 'bunion', 'pe...",6,13,49,19,"['wear', 'orthotic', 'support', 'arch', 'ortho...","['wear', 'orthotic', 'support', 'shoe', 'arch'...","['support', 'arch', 'fit', 'perfectly orthotic...","['fit', 'bunion', 'shoe']"


In [ ]:
val_df.head()

,review_ID,rating,vote,binaryrating,review,summary,cheat,cheat_num,overlap,review_len,summary_len,POS_keys,DEP_keys,Noun_adj_keys,TextRank_keys
111956,1448841600,1,10,negative,the package come orange box chinese lettering ...,<s> does not look anything like the amazon pri...,"{'prime', 'amazon', 'picture', 'like'}",4,9,50,11,"['chinese', 'picture', 'box', 'prime', 'letter...","['come', 'chinese', 'package', 'amazon', 'pict...","['package', 'amazon', 'picture', 'box', 'lette...","['orange', 'box', 'chinese', 'try']"
356131,1205366400,2,7,negative,really like this leather case but find one maj...,<s> nice case but major flaw with the leather ...,"{'major', 'belt', 'case', 'leather', 'clip', '...",6,9,60,13,"['flaw', 'major', 'case', 'clip', 'fall', 'tig...","['find', 'major', 'case', 'leather', 'flaw', '...","['leather', 'case', 'flaw', 'major', 'case', '...","['leather', 'case', 'pull', 'major']"
16534,1129766400,1,5,negative,have this recorder for about year . for first ...,<s> guy do not buy this panasonic dmr </s>,"{'buy', 'guy', 'panasonic'}",3,4,70,9,"['', 'first', 'mon', '', 'record', 'program', ...","['year', 'have', 'recorder', 'first', 'mon', '...","['mon', 'first', 'service', 'time', 'contact',...","['ago', 'start', 'problem', 'ask', 'recorder',..."
79723,1419811200,5,2,positive,when see how low the price was this did lot re...,<s> an excellent dealand here how to find out ...,"{'panel', 'tv', 'find'}",3,7,143,15,"['', 'great', 'average', 'black', 'home', 'fea...","['connectivity', 'hdmi', 'drive', 'has', 'vide...","['video', 'connectivity', 'black', 'watch', 'c...","['panel', 'samsung', 'webpage', 'great', 'manu..."
221481,1300752000,4,38,positive,order the red chair and far great . buy these ...,<s> seat does not fold in half </s>,"{'fold', 'seat', 'half'}",3,5,187,8,"['chair', 'red', '', '', 'little', 'chair', 'f...","['chair', 'order', 'red', 'little', 'bit', 'si...","['chair', 'red', 'chair', 'bit', 'foam', 'litt...","['red', 'chair', 'foam', 'similar', 'regular',..."


In [ ]:
p_train_df = analay(deepcopy(train_df), 'train')
p_train_df.to_excel('Analay/bert_train_df.xlsx', encoding='utf8')

p_val_df = analay(deepcopy(val_df), 'test')
p_val_df.to_excel('Analay/bert_val_df.xlsx', encoding='utf8')

train row: 430441 | overlap_pos_bert: 2 (43.48)percent_bert| len_lcs_bert: 10:  11%|█         | 48703/461643 [41:27<6:28:07, 17.73it/s]

In [ ]:
p_train_df.describe()

In [ ]:
import matplotlib.pyplot as plt
print('reveiw_len < 40 :',len(p_train_df[p_train_df['reveiw_len']<40]))
print('reveiw_len > 1200 :',len(p_train_df[p_train_df['reveiw_len']>1000])) # # max pos, max_enc_steps 
s = p_train_df['reveiw_len'].describe(); print(s)
s = s.drop(labels = ['count'])
s.plot.bar()
plt.ylabel('length')
plt.title('reveiw_len')
plt.show()

In [ ]:
print('summary_len > 40 :',len(p_train_df[p_train_df['summary_len']>40]))  # max_dec_steps
print('summary_len <= 10 :',len(p_train_df[p_train_df['summary_len']<=10])) # min_dec_steps

s = p_train_df['summary_len'].describe(); print(s)
s = s.drop(labels = ['count'])
s.plot.bar()
plt.ylabel('length')
plt.title('summary_len')
plt.show()

# percent_lcs EDA(20 divided)

In [ ]:
# 统计学上的四分为函数
# 原则上p是可以取0到1之间的任意值的。但是有一个四分位数是p分位数中较为有名的。

# 所谓四分位数；即把数值由小到大排列并分成四等份，处于三个分割点位置的数值就是四分位数。

# 第1四分位数 (Q1)，又称“较小四分位数”，等于该样本中所有数值由小到大排列后第25%的数字。
# 第2四分位数 (Q2)，又称“中位数”，等于该样本中所有数值由小到大排列后第50%的数字。
# 第3四分位数 (Q3)，又称“较大四分位数”，等于该样本中所有数值由小到大排列后第75%的数字。
# 第3四分位数与第1四分位数的差距又称四分位距（InterQuartile Range,IQR）

s = p_train_df['percent_lcs'].describe(); print(s)
s = s.drop(labels = ['count'])
s.plot.bar()
plt.ylabel('overlap(%)')
plt.title('percent_lcs')
plt.show()

print('percent_lcs x < 20 :%.2f' % float(len(p_train_df[p_train_df['percent_lcs']<20])/len(p_train_df)*100),'%' )  # overlap percent with summary
print('percent_lcs 20 < x < 40 :%.2f' % float( \
      len(p_train_df[(p_train_df['percent_lcs']>20) & (p_train_df['percent_lcs']<40)])/len(p_train_df)*100),'%')  # max_dec_steps
print('percent_lcs 40 < x < 60 :%.2f' % float( \
      len(p_train_df[(p_train_df['percent_lcs']>40) & (p_train_df['percent_lcs']<60)])/len(p_train_df)*100),'%')  # max_dec_steps
print('percent_lcs 60 < x < 80 :%.2f' % float( \
      len(p_train_df[(p_train_df['percent_lcs']>60) & (p_train_df['percent_lcs']<80)])/len(p_train_df)*100),'%')  # max_dec_steps
print('percent_lcs x > 80  :%.2f' % float( \
      len(p_train_df[(p_train_df['percent_lcs']>80)])/len(p_train_df)*100),'%')  # max_dec_steps


s = pd.Series({'x < 20':len(p_train_df[p_train_df['percent_lcs']<20])/len(p_train_df)*100, 
           '20 < x < 40':len(p_train_df[(p_train_df['percent_lcs']>20) & (p_train_df['percent_lcs']<40)])/len(p_train_df)*100, 
           '40 < x < 60':len(p_train_df[(p_train_df['percent_lcs']>40) & (p_train_df['percent_lcs']<60)])/len(p_train_df)*100, 
           '60 < x < 80':len(p_train_df[(p_train_df['percent_lcs']>60) & (p_train_df['percent_lcs']<80)])/len(p_train_df)*100, 
           'x > 80':len(p_train_df[p_train_df['percent_lcs']>80])/len(p_train_df)*100          
          })
s.plot.bar()
plt.ylabel('total(%)')
plt.title('percent_lcs EDA(20% divided)')
plt.show()

In [ ]:
'''
overlap => lcs > (20% summary len)
overlap_pos : 0 Top
overlap_pos : 1 Final
overlap_pos : 2 Other Pos
overlap_pos : -1 No overlap
'''

# print('overlap Top : %2.2f'% float(len(p_train_df[p_train_df['overlap_pos'] == 0])/len(p_train_df)*100),'%') 
# print('overlap Final :%2.2f'% float(len(p_train_df[p_train_df['overlap_pos'] == 1])/len(p_train_df)*100),'%') 
# print('overlap Other Pos :%2.2f'% float(len(p_train_df[p_train_df['overlap_pos'] == 2])/len(p_train_df)*100),'%')
# print('No overlap :%2.2f'% float(len(p_train_df[p_train_df['overlap_pos'] == -1])/len(p_train_df)*100),'%') 

# s = pd.Series({'overlap Top':len(p_train_df[p_train_df['overlap_pos'] == 0])/len(p_train_df)*100, 
#            'overlap Final':len(p_train_df[p_train_df['overlap_pos'] == 1])/len(p_train_df)*100, 
#            'overlap Other Pos':len(p_train_df[p_train_df['overlap_pos'] == 2])/len(p_train_df)*100, 
#            'No overlap':len(p_train_df[p_train_df['overlap_pos'] == -1])/len(p_train_df)*100, 
#           })
# print(s)
# s.plot.bar()
# plt.ylabel('overlap(%)')
# plt.title('percent_lcs')
# plt.show()

# Ext/Abs分界(50)

In [ ]:
print('percent_lcs x > 50  %2.2f'% float(len(p_train_df[(p_train_df['percent_lcs']>50)])/len(p_train_df)*100),'%')  # max_dec_steps
print('percent_lcs x <= 50  %2.2f'% float(len(p_train_df[(p_train_df['percent_lcs']<=50)])/len(p_train_df)*100),'%')  # max_dec_steps

s = pd.Series({'x > 50':len(p_train_df[(p_train_df['percent_lcs']>50)])/len(p_train_df)*100, 
           'x < 50':len(p_train_df[(p_train_df['percent_lcs']<=50)])/len(p_train_df)*100,      
          })
s.plot.bar()
plt.ylabel('total(%)')
plt.title('percent_lcs')
plt.show()

In [ ]:
'''
overlap => lcs > (20% summary len)
overlap_pos : 0 Top
overlap_pos : 1 Final
overlap_pos : 2 Other Pos
overlap_pos : -1 No overlap
'''

print('overlap Top :%2.2f'% float(len(p_train_df[(p_train_df['overlap_pos'] == 0) & (p_train_df['percent_lcs']>50)])/len(p_train_df)*100),'%') 
print('overlap Final :%2.2f'% float(len(p_train_df[(p_train_df['overlap_pos'] == 1) & (p_train_df['percent_lcs']>50)])/len(p_train_df)*100),'%') 
print('overlap Other Pos :%2.2f'% float(len(p_train_df[(p_train_df['overlap_pos'] == 2) & (p_train_df['percent_lcs']>50)])/len(p_train_df)*100),'%') 
print('No overlap :%2.2f'% float(len(p_train_df[(p_train_df['percent_lcs']<=50)])/len(p_train_df)*100),'%') 

s = pd.Series({'overlap Top':len(p_train_df[(p_train_df['overlap_pos'] == 0) & (p_train_df['percent_lcs']>50)])/len(p_train_df)*100, 
           'overlap Final':len(p_train_df[(p_train_df['overlap_pos'] == 1) & (p_train_df['percent_lcs']>50)])/len(p_train_df)*100,      
           'overlap Other Pos':len(p_train_df[(p_train_df['overlap_pos'] == 2) & (p_train_df['percent_lcs']>50)])/len(p_train_df)*100,  
           'No overlap':len(p_train_df[(p_train_df['percent_lcs']<=50)])/len(p_train_df)*100,  
          })
s.plot.bar()
plt.ylabel('total(%)')
plt.title('overlap')
plt.show()

In [ ]:
def view(df, condition, view_num=10):
    package_dict = []
    for i in range(view_num):
        d1 = dict(p_train_df[eval(condition)].iloc[i])
        d2 = {'review':d1['review'],
              'summary':d1['summary'],
              'token_lcs':d1['token_lcs'],
              'overlap':d1['overlap'],
              'percent_lcs':d1['percent_lcs'],
             }
        package_dict.append(d2)
    return package_dict
        
# view(p_train_df, "(p_train_df['overlap_pos'] == 2) & (p_train_df['percent_lcs']>50)")
view(p_train_df, "(p_train_df['summary_len']<11)")

In [ ]:
dict(p_train_df[(p_train_df['overlap_pos'] == 2) & (p_train_df['percent_lcs']>50)].iloc[0:10])['summary']